# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [396]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. '''

## 2. Ask for parameters

In [397]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [398]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*  $i = 1, 2$: Index for the two different raw materials.
*  $j = 1, 2$: Index for the two different drugs.

## Parameters:

*  $P_j$: Selling price per unit of drug $j$.
*  $C_i$: Purchasing cost per unit of raw material $i$.
*  $OC_j$: Operational cost per unit of drug $j$ produced.
*  $B$: Total budget for purchasing raw materials and operating the production process.
*  $Cap_j$: Maximum production capacity for drug $j$.
*  $S_i$: Maximum storage capacity for raw material $i$.
*  $RA_j$: Required amount of active ingredient in drug $j$.
*  $\alpha_i$: Nominal amount of active ingredient extracted per unit of raw material $i$. 
*  $\beta_i$: Maximum percentage variation in the extraction of active ingredient from raw material $i$ (i.e., 0.005 for raw material 1 and 0.02 for raw material 2).

## Decision Variables:

*  $x_i$: Amount of raw material $i$ to purchase.
*  $y_j$: Amount of drug $j$ to produce. 


# 2. Ask for objective

In [399]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [400]:
Markdown(response2.text)

## Objective Function:

Maximize:  $\sum_{j=1}^{2} P_j y_j - \sum_{i=1}^{2} C_i x_i - \sum_{j=1}^{2} OC_j y_j $ 


# 3. Ask for constraints

In [401]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [402]:
Markdown(response3.text)

## Constraints:

1. **Budget Constraint:**
   $$ \sum_{i=1}^{2} C_i x_i + \sum_{j=1}^{2} OC_j y_j \leq B $$

2. **Capacity Constraints:**
   $$ y_j \leq Cap_j,  \quad  \forall j = 1, 2 $$

3. **Storage Constraints:**
   $$ x_i \leq S_i, \quad  \forall i = 1, 2 $$

4. **Active Ingredient Constraint (Worst-Case Scenario):**
   $$ \sum_{i=1}^{2} \alpha_i (1 - \beta_i) x_i \geq \sum_{j=1}^{2} RA_j y_j $$ 

5. **Non-Negativity Constraints:**
   $$ x_i \geq 0, \quad \forall i = 1, 2 $$
   $$ y_j \geq 0, \quad \forall j = 1, 2 $$ 


## 3. Generate the pyomo code

In [403]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [404]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data - Replace with your actual data
P = {1: 100, 2: 150}  # Selling price per unit of drug j
C = {1: 20, 2: 30}  # Purchasing cost per unit of raw material i
OC = {1: 10, 2: 15}  # Operational cost per unit of drug j produced
B = 10000  # Total budget
Cap = {1: 500, 2: 300}  # Maximum production capacity for drug j
S = {1: 800, 2: 1200}  # Maximum storage capacity for raw material i
RA = {1: 0.5, 2: 0.8}  # Required amount of active ingredient in drug j
alpha = {1: 0.6, 2: 0.4}  # Nominal amount of active ingredient extracted per unit of raw material i
beta = {1: 0.005, 2: 0.02}  # Maximum percentage variation in the extraction

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Define parameters
model.P = pyo.Param(model.j, initialize=P)
model.C = pyo.Param(model.i, initialize=C)
model.OC = pyo.Param(model.j, initialize=OC)
model.B = pyo.Param(initialize=B)
model.Cap = pyo.Param(model.j, initialize=Cap)
model.S = pyo.Param(model.i, initialize=S)
model.RA = pyo.Param(model.j, initialize=RA)
model.alpha = pyo.Param(model.i, initialize=alpha)
model.beta = pyo.Param(model.i, initialize=beta)

# Define variables
model.x = pyo.Var(model.i, within=pyo.NonNegativeReals)  # Amount of raw material i to purchase
model.y = pyo.Var(model.j, within=pyo.NonNegativeReals)  # Amount of drug j to produce

# Define the objective function
def objective_rule(model):
    return sum(model.P[j] * model.y[j] for j in model.j) - sum(model.C[i] * model.x[i] for i in model.i) - sum(model.OC[j] * model.y[j] for j in model.j)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def budget_constraint_rule(model):
    return sum(model.C[i] * model.x[i] for i in model.i) + sum(model.OC[j] * model.y[j] for j in model.j) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint_rule)

def capacity_constraint_rule(model, j):
    return model.y[j] <= model.Cap[j]
model.capacity_constraint = pyo.Constraint(model.j, rule=capacity_constraint_rule)

def storage_constraint_rule(model, i):
    return model.x[i] <= model.S[i]
model.storage_constraint = pyo.Constraint(model.i, rule=storage_constraint_rule)

def active_ingredient_constraint_rule(model):
    return sum(model.alpha[i] * (1 - model.beta[i]) * model.x[i] for i in model.i) >= sum(model.RA[j] * model.y[j] for j in model.j)
model.active_ingredient_constraint = pyo.Constraint(rule=active_ingredient_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Purchase Raw Material 1:", model.x[1].value)
print("Purchase Raw Material 2:", model.x[2].value)
print("Produce Drug 1:", model.y[1].value)
print("Produce Drug 2:", model.y[2].value)
print("Total Profit:", model.profit.value)
```


## 4. Input problem data and try running the generated code

In [406]:
import pyomo.environ as pyo

# Sample data - Replace with your actual data
P = {1: 100, 2: 150}  # Selling price per unit of drug j
C = {1: 20, 2: 30}  # Purchasing cost per unit of raw material i
OC = {1: 10, 2: 15}  # Operational cost per unit of drug j produced
B = 10000  # Total budget
Cap = {1: 500, 2: 300}  # Maximum production capacity for drug j
S = {1: 800, 2: 1200}  # Maximum storage capacity for raw material i
RA = {1: 0.5, 2: 0.8}  # Required amount of active ingredient in drug j
alpha = {1: 0.6, 2: 0.4}  # Nominal amount of active ingredient extracted per unit of raw material i
beta = {1: 0.005, 2: 0.02}  # Maximum percentage variation in the extraction

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Define parameters
model.P = pyo.Param(model.j, initialize=P)
model.C = pyo.Param(model.i, initialize=C)
model.OC = pyo.Param(model.j, initialize=OC)
model.B = pyo.Param(initialize=B)
model.Cap = pyo.Param(model.j, initialize=Cap)
model.S = pyo.Param(model.i, initialize=S)
model.RA = pyo.Param(model.j, initialize=RA)
model.alpha = pyo.Param(model.i, initialize=alpha)
model.beta = pyo.Param(model.i, initialize=beta)

# Define variables
model.x = pyo.Var(model.i, within=pyo.NonNegativeReals)  # Amount of raw material i to purchase
model.y = pyo.Var(model.j, within=pyo.NonNegativeReals)  # Amount of drug j to produce

# Define the objective function
def objective_rule(model):
    return sum(model.P[j] * model.y[j] for j in model.j) - sum(model.C[i] * model.x[i] for i in model.i) - sum(model.OC[j] * model.y[j] for j in model.j)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def budget_constraint_rule(model):
    return sum(model.C[i] * model.x[i] for i in model.i) + sum(model.OC[j] * model.y[j] for j in model.j) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint_rule)

def capacity_constraint_rule(model, j):
    return model.y[j] <= model.Cap[j]
model.capacity_constraint = pyo.Constraint(model.j, rule=capacity_constraint_rule)

def storage_constraint_rule(model, i):
    return model.x[i] <= model.S[i]
model.storage_constraint = pyo.Constraint(model.i, rule=storage_constraint_rule)

def active_ingredient_constraint_rule(model):
    return sum(model.alpha[i] * (1 - model.beta[i]) * model.x[i] for i in model.i) >= sum(model.RA[j] * model.y[j] for j in model.j)
model.active_ingredient_constraint = pyo.Constraint(rule=active_ingredient_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Purchase Raw Material 1:", model.x[1].value)
print("Purchase Raw Material 2:", model.x[2].value)
print("Produce Drug 1:", model.y[1].value)
print("Produce Drug 2:", model.y[2].value)
print("Total Profit:", model.profit())

Optimal Solution:
Purchase Raw Material 1: 313.087038196619
Purchase Raw Material 2: 0.0
Produce Drug 1: 373.825923606763
Produce Drug 2: 0.0
Total Profit: 27382.592360676295


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [407]:
print(response.text)

## Indices:

*  $i = 1, 2$: Index for the two different raw materials.
*  $j = 1, 2$: Index for the two different drugs.

## Parameters:

*  $P_j$: Selling price per unit of drug $j$.
*  $C_i$: Purchasing cost per unit of raw material $i$.
*  $OC_j$: Operational cost per unit of drug $j$ produced.
*  $B$: Total budget for purchasing raw materials and operating the production process.
*  $Cap_j$: Maximum production capacity for drug $j$.
*  $S_i$: Maximum storage capacity for raw material $i$.
*  $RA_j$: Required amount of active ingredient in drug $j$.
*  $\alpha_i$: Nominal amount of active ingredient extracted per unit of raw material $i$. 
*  $\beta_i$: Maximum percentage variation in the extraction of active ingredient from raw material $i$ (i.e., 0.005 for raw material 1 and 0.02 for raw material 2).

## Decision Variables:

*  $x_i$: Amount of raw material $i$ to purchase.
*  $y_j$: Amount of drug $j$ to produce. 



In [408]:
print(response2.text)

## Objective Function:

Maximize:  $\sum_{j=1}^{2} P_j y_j - \sum_{i=1}^{2} C_i x_i - \sum_{j=1}^{2} OC_j y_j $ 



In [409]:
print(response3.text)

## Constraints:

1. **Budget Constraint:**
   $$ \sum_{i=1}^{2} C_i x_i + \sum_{j=1}^{2} OC_j y_j \leq B $$

2. **Capacity Constraints:**
   $$ y_j \leq Cap_j,  \quad  \forall j = 1, 2 $$

3. **Storage Constraints:**
   $$ x_i \leq S_i, \quad  \forall i = 1, 2 $$

4. **Active Ingredient Constraint (Worst-Case Scenario):**
   $$ \sum_{i=1}^{2} \alpha_i (1 - \beta_i) x_i \geq \sum_{j=1}^{2} RA_j y_j $$ 

5. **Non-Negativity Constraints:**
   $$ x_i \geq 0, \quad \forall i = 1, 2 $$
   $$ y_j \geq 0, \quad \forall j = 1, 2 $$ 



In [410]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data - Replace with your actual data
P = {1: 100, 2: 150}  # Selling price per unit of drug j
C = {1: 20, 2: 30}  # Purchasing cost per unit of raw material i
OC = {1: 10, 2: 15}  # Operational cost per unit of drug j produced
B = 10000  # Total budget
Cap = {1: 500, 2: 300}  # Maximum production capacity for drug j
S = {1: 800, 2: 1200}  # Maximum storage capacity for raw material i
RA = {1: 0.5, 2: 0.8}  # Required amount of active ingredient in drug j
alpha = {1: 0.6, 2: 0.4}  # Nominal amount of active ingredient extracted per unit of raw material i
beta = {1: 0.005, 2: 0.02}  # Maximum percentage variation in the extraction

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Define parameters
model.P = pyo.Param(model.j, initialize=P)
model.C = pyo.Param(model.i, initialize=C)
model.OC = pyo.Param(model.j, initiali